In [0]:
import pyspark.sql.functions as F 

In [0]:
# 1. Definir la ruta de los archivos de entrada (ajústala a tu almacenamiento)
input_paths = [
    "/Volumes/dbdemos/default/test_data/parte_1.txt",
    "/Volumes/dbdemos/default/test_data/parte_2.txt",
    "/Volumes/dbdemos/default/test_data/parte_3.txt"
]

# 2. Función para leer archivos TSV con esquema inferido
def read_tsv(path: str):
    return (spark.read
            .option("header", True)   # si los archivos tienen encabezado
            .option("sep", "\t")      # tabulador
            .option("inferSchema", True)
            .csv(path))

# 3. Leer y unir los tres archivos
dfs = [read_tsv(path) for path in input_paths]
df_consolidado = dfs[0].unionByName(dfs[1], allowMissingColumns=True) \
                       .unionByName(dfs[2], allowMissingColumns=True)

# 4. Ejemplo de uso de funciones F (por buenas prácticas en Spark)
# 👉 Se agrega una columna con la fecha de carga
df_consolidado = df_consolidado.withColumn("fecha_carga", F.current_date())

# 5. Guardar en tabla del catálogo (formato Delta recomendado en Silver Layer)
(
    df_consolidado.write
    .format("delta")
    .mode("overwrite")  # cuidado: sobreescribe la tabla
    .saveAsTable("dbdemos.default.colaboradores_consolidados")
)

print("✅ Consolidación completada y tabla creada en dbdemos.default.colaboradores_consolidados")


In [0]:
%sql
SELECT *
FROM dbdemos.default.colaboradores_consolidados;